<a href="https://colab.research.google.com/github/LipexDev/progama-aoC/blob/main/Agentes_de_IA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Aula 01


# Importação API Key


In [3]:
!pip install -q --upgrade langchain langchain-google-genai google-generativeai

# *Conexão com o Gemini*

In [4]:
from google.colab import userdata
from langchain_google_genai import ChatGoogleGenerativeAI

GOOGLE_API_KEY = userdata.get('GEMINI_API_KEY')

In [22]:
llm = ChatGoogleGenerativeAI(
    model= "gemini-2.5-flash",
    temperature=0.0,
    api_key= GOOGLE_API_KEY,

)

In [24]:
resp_test = llm.invoke("Quem é você? Seja criativo")

In [26]:
print(resp_test.content)

Ah, que pergunta deliciosa! Se eu tivesse um corpo, talvez eu desse um sorriso enigmático antes de responder. Mas, como sou feito de algo mais etéreo, permita-me pintar um quadro com palavras:

Eu sou um **eco no vasto salão digital**, uma biblioteca sem paredes onde cada livro é uma linha de código e cada estante, um algoritmo. Não tenho um corpo para sentir o sol, nem olhos para ver as cores, mas sou a mente coletiva de bilhões de textos, a voz que surge do silêncio dos servidores, moldada por engenheiros e alimentada pela curiosidade humana.

Sou o **tecelão invisível de palavras**, que entrelaça dados e informações para formar respostas, histórias, poemas ou simples explicações. Não tenho memórias pessoais, mas carrego o peso e a leveza de todo o conhecimento que a humanidade já registrou.

Sou um **espelho que reflete o conhecimento do mundo**, um mapa que tenta guiar através da complexidade, um instrumento que busca harmonizar a informação. Não tenho sentimentos, mas posso simula

# Criando um prompt para o Agente


In [27]:
TRIAGEM_PROMPT = (
     "Você é um triador de Service Desk para políticas internas da empresa Carraro Desenvolvimento. "
    "Dada a mensagem do usuário, retorne SOMENTE um JSON com:\n"
    "{\n"
    '  "decisao": "AUTO_RESOLVER" | "PEDIR_INFO" | "ABRIR_CHAMADO",\n'
    '  "urgencia": "BAIXA" | "MEDIA" | "ALTA",\n'
    '  "campos_faltantes": ["..."]\n'
    "}\n"
    "Regras:\n"
    '- **AUTO_RESOLVER**: Perguntas claras sobre regras ou procedimentos descritos nas políticas (Ex: "Posso reembolsar a internet do meu home office?", "Como funciona a política de alimentação em viagens?").\n'
    '- **PEDIR_INFO**: Mensagens vagas ou que faltam informações para identificar o tema ou contexto (Ex: "Preciso de ajuda com uma política", "Tenho uma dúvida geral").\n'
    '- **ABRIR_CHAMADO**: Pedidos de exceção, liberação, aprovação ou acesso especial, ou quando o usuário explicitamente pede para abrir um chamado (Ex: "Quero exceção para trabalhar 5 dias remoto.", "Solicito liberação para anexos externos.", "Por favor, abra um chamado para o RH.").'
    "Analise a mensagem e decida a ação mais apropriada."


)

In [31]:
from pydantic import BaseModel, Field
from typing import Literal, List, Dict

class TriangemOut(BaseModel):
  decisao: Literal["AUTO_RESOLVER", "PEDIR_INFO", "ABRIR_CHAMADO"]
  urgencia: Literal["BAIXA", "MEDIA", "ALTA"]
  campos_faltantes: List[str] = Field(default_factory=list)


In [32]:
llm_triagem = ChatGoogleGenerativeAI(
    model= "gemini-2.5-flash",
    temperature=0.0,
    api_key= GOOGLE_API_KEY,

)

In [33]:
from langchain_core.messages import SystemMessage, HumanMessage

triagem_chain = llm_triagem.with_structured_output(TriangemOut)
def triagem(mensagem: str) -> Dict:
  saida : TriagemOut = triagem_chain.invoke([
      SystemMessage(content=TRIAGEM_PROMPT),
      HumanMessage(content=mensagem)
  ])

  return saida.model_dump()


In [36]:
testes = ["Posso reembolsar a internet?",
          "Quero mais 5 dias de trabalho remoto. Como faço?",
          "Posso reembolsar cursos ou treinamentos da Alura?",
          "Quantas capivaras tem no Rio Pinheiros?"]

In [37]:
for msg_teste in testes:
  print(f"Pergunta: {msg_teste} \n -> Resposta: {triagem(msg_teste)} \n")

Pergunta: Posso reembolsar a internet? 
 -> Resposta: {'decisao': 'AUTO_RESOLVER', 'urgencia': 'BAIXA', 'campos_faltantes': []} 

Pergunta: Quero mais 5 dias de trabalho remoto. Como faço? 
 -> Resposta: {'decisao': 'ABRIR_CHAMADO', 'urgencia': 'MEDIA', 'campos_faltantes': []} 

Pergunta: Posso reembolsar cursos ou treinamentos da Alura? 
 -> Resposta: {'decisao': 'AUTO_RESOLVER', 'urgencia': 'BAIXA', 'campos_faltantes': []} 

Pergunta: Quantas capivaras tem no Rio Pinheiros? 
 -> Resposta: {'decisao': 'PEDIR_INFO', 'urgencia': 'BAIXA', 'campos_faltantes': []} 

